# 4장 karplus_strong

음악 소리의 주요 특징 중 하나는 음고(주파수)다. 
주파수는 초당 진동 수를 가리키며, 단위는 헤르츠[Hz]다

예를 들어, 어쿠스틱 기타의 위쪽에서 세번째 줄은 146.83Hz의 주파수로 D음을 내는데, </br>
이 소리는 컴퓨터에서 146.83Hz의 주파수를 갖는 정현파를 생성함으로써 비슷하게 만들어 낼 수 있다. 

실제 기타 음은 다양한 강도의 혼합 주파수로 만들어 진다.</br> 
배음(倍音)은 하나의 음을 구성하는 여러 부분음들 중, 기본음(基本音)보다 높은 정수배의 진동수를 갖는 모든 상음(上音)들을 가리키는 말이다.

## 0. 카플러스 스트롱 알고리즘
따라서, 컴퓨터에서 현악기에서 만들어진 소리를 시뮬레이션 하기 위해서는 기본 주파수와 배음을 모두 생성해야 하는데, 
이때 필요한 알고리즘이 "카플러스 스트롱" 알고리즘 이다. 

## 1. 목표
- 카플러스 스트롱알고리즘을 사용하여 기타 음 5개를 생성한다. 
- 그리고 이 음들을 생성하는 데 사용된 알고리즘을 시각화하고, WAV파일로 저장한다. 
- 또한 임의로 재생하는 방법을 작성하고, 다음 작업을 수행하는 방법을 배운다. 

1. 파이썬 deque 클래스를 사용해 원형 버퍼 구현하기
2. numpy배열과 ufuncs의 사용
3. pygame을 사용해 WAV 파일 재생하기
4. matplotlib을 사용해 그래프 그리기
5. 5음 음계 연주하기

## 2. 동작 원리
- 양단에 견고하게 묶여 있는 줄을 시뮬레이션하기 위해 변위 값으로 이뤄진 원형 버퍼(ring buffer)를 이용한다. 
- 원형 버퍼(순환버퍼, 링버퍼): 고정길이의 버퍼로서 스스로 순환하는 자료구조 이다. 원형버퍼의 끝에 도달했을 때, 그 다음에 접근되는 요소는 원형버퍼의 첫번째 요소 이다. 
- 원형버퍼의 길이(N)는 N = S/f에 의해 정해지는데, S는 샘플링 레이트(sampling rate)이고 f는 주파수(frequency)다. 
- 원형버퍼는 deque(deck)을 이요해 구현한다.

### 2-1 원형버퍼 구현하기

In [12]:
from collections import deque
d = deque(range(10))
print(type(d))
print(d) 

<class 'collections.deque'>
deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [13]:
d.append(-1)
print(d)

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, -1])


In [14]:
d.popleft()

0

In [15]:
print(d)

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, -1])


In [30]:
a = list(range(10))
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [31]:
a.pop(0)

0

In [32]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [33]:
a.append(-1)

In [34]:
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, -1]

### 2-2 카플러스 스트롱 알고리즘 구현하기
- 오디오 데이터 생성

In [4]:
# 주어진 주파수 음을 생성한다
def generateNote(freq):
    nSamples = 44100
    sampleRate = 44100
    N = int(sampleRate/freq)
    #원형 버퍼를 초기화 한다
    buf = deque([random.random() - 0.5 for i in range(N)])
    #샘플 버퍼를 초기화 한다.
    samples = np.array([0]*nSamples, 'float32')
    for i in range(nSamples):
        samples[i] = buf[0]
        avg = 0.996*0.5*(buf[0] + buf[1])
        buf.append(avg)
        buf.popleft()
    
    #샘플을 16비트 값으로 반환하고, 다시 문자열로 변환한다.
    #최댓값은 32767이다. 
    samples = np.array(samples*32767, 'int16')
    return samples.tostring()

### 2-3 WAV파일 기록하기
- 오디오 데이터가 생성되었으니 파이썬 wave모듈을 사용해 WAV파일에 기록할 수 있다.

In [5]:
def writeWAVE(fname, data):
    # 파일을 연다
    file = wave.open(fname, 'wb')
    # WAV파일을 위한 매개변수들..
    nChannels = 1
    sampleWidth = 2
    frameRate = 44100
    nFrames = 44100
    # 매개변수들을 설정한다.
    file.setparams((nchannels, sampleWidth, frameRate, nFrames, 'NONE', 'noncompressed'))
    file.writeframes(data)
    file.close()

### 2-4 pygame으로 WAV파일 재생하기
- 이번에는 알고리즘에 의해 생성된 WAV파일을 재생하기 위해 파이썬 pygame모듈을 사용해 본다. 

In [ ]:
#WAV 파일을 재생한다
class NotePlayer:
    # 생성자
    def __init__(self):
        pygame.mixer.pre_init(44100, -16, 1, 2048)
        pygame.init()
        # 음으로 이뤄진 딕셔너리
        self.notes = {}
    # 음을 추가한다
    def add(self, fileName):
        self.notes[fileName] = pygame.mixer.Sound(fileName)
    # 음을 재생한다
    def play(self, fileName):
        try:
            self.notes[fileName].play()
        except:
            print(fileName + ' not found! ')
    def playRandom(self):
        """음을 임의로 재생한다. """
        index = random.randint(0, len(self.notes)-1)
        note = list(self.notes.values())[index]
        note.play()
        

### 2-5 main() 메소드

In [9]:
import argparse
import parser

parser = argparse.ArgumentParser(description="Generating sounds with Karplus String Algorithm")
# 인수들을 추가한다
parser.add_argument('--display', action='store_true', required=False)
parser.add_argument('--play', action='store_true', required=False)
parser.add_argument('--piano', action='store_true', required=False)
args = parser.parse_args()

# 플래그가 설정되어 있으면 플롯을 보여준다.
if args.display:
    dShowPlot = True
    plt.ion()

# 플레이어(음 재생기)를 생성한다.
nplayer = NotePlayer()

print('creaing notes...')
for name, freq in list(pmNotes.items()):
    fileName = name + '.wav'
    if not os.path.exists(fileName) or args.display:
        data = generateNote(freq)
        print('creating ' + fileName + '...')
        writeWAVE(fileName, data)
    else:
        print('fileName already created. skipping...')
    
    # 플레이어에 음을 추가한다. 
    nplayer.add(name + '.wav')

    # 디스플레이 플래그가 설정되어 있으면 음을 재생한다. 
    if args.display:
        nplayer.play(name + '.wav')
        time.sleep(0.5)
    
    # 임의로 음을 재생한다.
    if args.play:
        while True:
            try:
                nplayer.palyRandom()
                # 휴지 - 1부터 8비트
                rest = np.ramdom.choice([1, 2, 4, 8], 1,
                                        p = [0.15, 0.7, 0.1, 0.05])
                time.sleep(0.25*rest[0])
            except KeyboardInterrupt:
                exit()

usage: ipykernel_launcher.py [-h] [--display] [--play] [--piano]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/jooyoungson/Library/Jupyter/runtime/kernel-96827128-554b-4391-a236-ef03addc4f27.json


SystemExit: 2

## 6. 전체 코드

https://github.com/electronut/pp/blob/master/karplus/ks.py